# 📊 Airbnb: Exploratory Data Analysis

This notebook explores the outputs of our Spark-based Airbnb data pipeline.  
We will analyze estimated revenue, occupancy rates, and trends across neighborhoods.

---

✅ Data sources:
- `cleaned_listings.parquet`
- `cleaned_calendar.parquet`
- `revenue_by_listing.csv`
- `revenue_by_neighborhood.csv`

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Create Spark session
spark = SparkSession.builder.appName("AirbnbEDA").getOrCreate()

In [ ]:
# Load cleaned listings
listings_df = spark.read.parquet("../output/cleaned_listings.parquet")

# Load cleaned calendar
calendar_df = spark.read.parquet("../output/cleaned_calendar.parquet")

# Load revenue by listing
revenue_listing_df = spark.read.option("header", True).csv("../output/revenue_by_listing.csv", inferSchema=True)

# Load revenue by neighborhood
revenue_neigh_df = spark.read.option("header", True).csv("../output/revenue_by_neighbourhood.csv", inferSchema=True)

In [ ]:
print("🧾 Listings:")
listings_df.show(5)

print("📅 Calendar:")
calendar_df.show(5)

print("💰 Revenue by Listing:")
revenue_listing_df.show(5)

print("📍 Revenue by Neighborhood:")
revenue_neigh_df.show(5)

In [ ]:
top_listings = revenue_listing_df.orderBy("total_revenue", ascending=False).limit(10)
top_listings.show(truncate=False)

In [ ]:
# Convert to Pandas for plotting
occupancy_pd = revenue_listing_df.select("occupancy_rate").toPandas()

plt.figure(figsize=(10,6))
sns.histplot(occupancy_pd["occupancy_rate"], bins=20, kde=True, color='skyblue')
plt.title("Distribution of Occupancy Rates")
plt.xlabel("Occupancy Rate")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Convert neighborhood data to Pandas
neigh_pd = revenue_neigh_df.toPandas()

# Sort by total revenue
neigh_pd_sorted = neigh_pd.sort_values(by="total_revenue", ascending=False)

# Create bar plot
plt.figure(figsize=(12,6))
sns.barplot(
    data=neigh_pd_sorted,
    x="listing_neighbourhood",
    y="total_revenue",
    palette="viridis"
)

plt.xticks(rotation=45, ha='right')
plt.title("Total Revenue by Neighborhood")
plt.ylabel("Total Revenue ($)")
plt.xlabel("Neighborhood")
plt.tight_layout()
plt.show()

In [ ]:
# Join listings with revenue and select only needed columns
listings_revenue = listings_df.join(
    revenue_listing_df,
    listings_df.id == revenue_listing_df.listing_id,
    how="inner"
).select(
    listings_df["room_type"],
    revenue_listing_df["total_revenue"]
)

# Group by room type and calculate average revenue
roomtype_df = listings_revenue.groupBy("room_type").avg("total_revenue")
roomtype_df = roomtype_df.withColumnRenamed("avg(total_revenue)", "avg_revenue")

roomtype_df.show()

# Plot the data
roomtype_pd = roomtype_df.toPandas()

sns.barplot(data=roomtype_pd, x="room_type", y="avg_revenue", palette="pastel")
plt.title("Average Revenue by Room Type")
plt.xlabel("Room Type")
plt.ylabel("Avg Revenue ($)")
plt.tight_layout()
plt.show()

In [ ]:
spark.stop()